In [1]:
from pathlib import Path
import pytest
import bw2calc as bc
import bw2data as bd
from bw_timex import TimexLCA
from datetime import datetime

bd.projects.set_current("__test_nonunitary__")
bd.Database("bio").write(
    {
        ("bio", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    },
)
bd.Database("db_2020").write(
    {
        ("db_2020", "C"): {
            "name": "c",
            "location": "somewhere",
            "reference product": "c",
            "exchanges": [
                {
                    "amount": 3, #produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 0.5,  
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ]
        },
}
)

bd.Database("foreground").write(
    { 
        ("foreground", "A"): {
            "name": "a",
            "location": "somewhere",
            "reference product": "a",
            "exchanges": [
                {
                    "amount": 1,
                    "type": "production",
                    "input": ("foreground", "A"),
                },
                {
                    "amount": 1.5,
                    "type": "technosphere",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 4,
                    "type": "technosphere",
                    "input": ("foreground", "B"),
                },
            ]
        },

        ("foreground", "B"): {
            "name": "b",
            "location": "somewhere",
            "reference product": "b",
            "exchanges": [
                {
                    "amount": 7, #produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("foreground", "B"),
                },
                {
                    "amount": 0.9,  
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ]
        }
    }
)

bd.Method(("GWP", "example")).write(
    [
        (("bio", "CO2"), 1),
    ]
)

100%|██████████| 1/1 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 932.69it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 4140.48it/s]


Vacuuming database 


In [2]:
bd.projects.set_current("__test_nonunitary__")

node_a = bd.get_node(database="foreground", code="A")

database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y")    ,         
    "foreground": "dynamic",  
}

tlca = TimexLCA(demand={node_a: 1}, method=("GWP", "example"), database_date_dict = database_date_dict)

tlca.build_timeline()
tlca.lci()
tlca.static_lcia()


Starting graph traversal
Calculation count: 2


c:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:182: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
c:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:444: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(


In [3]:
expected_score = (1 * 1.5 / 3 * 0.5 + # A -> Nonunitary in bd_2020
                  1 * 4 / 7 * 0.9) # A -> B -> Nonunitary in foreground 


expected_score 

0.7642857142857142

In [4]:
tlca.score

3.85

In [6]:
false_score =  (1 * 1.5 / 3 * 0.5 + # A -> Nonunitary in bd_2020
                  1 * 4  * 0.9) # A -> B -> Nonunitary in foreground (not scaled by production amount!)
false_score 

3.85